# Vector Space Models

This chapter discusses computational models to represent text in vector spaces.

* [Bag-of-Words](#Bag-of-Words)
* [Frequency-Counts](#Frequency-Counts)


## References

* [Vector Space Model](https://en.wikipedia.org/wiki/Vector_space_model)
* [Bag-of-Words Model](https://en.wikipedia.org/wiki/Bag-of-words_model)

## Bag-of-Words

Let there be a giant bag that can hold all unique words in the world.
Then, each token in a text such as "*Jinho Choi is a professor at Emory University .*" can be inserted to the bag as follows:

<img src="res/bow.jpg" width=600 align="left"/>

This bag can be represented by a vector of which every dimension stands for a unique token in the world.
All dimensions are initialized to `0`, except for the ones representing tokens in the input text, which are assigned with `1`:

<img src="res/vsm.jpg" width=600 align="left"/>

* What is the total dimension of this vector?
* How should we declare this vector in Python?
* Does this vector correctly represent the original text (anything missing)?

## Frequency Counts

Consider the following two documents:

```
D1: John bought a car . The car was fancy .
D2: Mary liked the car .  John gave it to Mary .
```

A **term frequency** (`tf`) is the number of occurrences of a specific term in a specific document:

```
tf(John, D1) = 1
tf(John, D2) = 1
tf(Mary, D2) = 2
```

A **document frequency** (`df`) is the number of documents containing a specific term:

```
df(John) = 2
df(John) = 2
```

In [1]:
!pip install requests
import requests

def download(remote_addr: str, local_addr: str):
    r = requests.get(remote_addr)
    fin = open(local_addr, 'wb')
    fin.write(r.content)

  Using cached requests-2.27.1-py2.py3-none-any.whl (63 kB)


In [3]:
aesop_link = 'https://raw.githubusercontent.com/emorynlp/data-collection/master/aesops_fables/dat/aesopfables.json?token=GHSAT0AAAAAABQLGADAWZAYHXOW7LMGBEAIYPZKZSA'
aesop_file = 'aesopfables.json'
download(aesop_link, aesop_file)

> Make sure which directory `aesopfables.json` is downloaded.